# Задача 1: Стек и Менеджер задач

In [1]:
class Stack:
    def __init__(self):
        self.items = []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        if not self.is_empty():
            return self.items.pop()
        return None

    def is_empty(self):
        return len(self.items) == 0

    def size(self):
        return len(self.items)

    def peek(self):
        if not self.is_empty():
            return self.items[-1]
        return None


class TaskManager:
    def __init__(self):
        self.tasks = {}

    def new_task(self, task, priority):
        if priority not in self.tasks:
            self.tasks[priority] = Stack()
        self.tasks[priority].push(task)

    def remove_task(self, task):
        for priority in self.tasks:
            temp_stack = Stack()
            found = False
            while not self.tasks[priority].is_empty():
                current_task = self.tasks[priority].pop()
                if current_task == task and not found:
                    found = True
                    continue
                temp_stack.push(current_task)
            while not temp_stack.is_empty():
                self.tasks[priority].push(temp_stack.pop())
            if found:
                break

    def __str__(self):
        result = []
        sorted_priorities = sorted(self.tasks.keys())
        for priority in sorted_priorities:
            tasks_list = []
            temp_stack = Stack()
            while not self.tasks[priority].is_empty():
                temp_stack.push(self.tasks[priority].pop())
            while not temp_stack.is_empty():
                task = temp_stack.peek()
                tasks_list.append(task)
                self.tasks[priority].push(temp_stack.pop())
            if tasks_list:
                result.append(f"{priority} {'; '.join(tasks_list)}")
        return '\n'.join(result)


manager = TaskManager()
manager.new_task("сделать уборку", 4)
manager.new_task("помыть посуду", 4)
manager.new_task("отдохнуть", 1)
manager.new_task("поесть", 2)
manager.new_task("сдать дз", 2)
print(manager)

1 отдохнуть
2 поесть; сдать дз
4 сделать уборку; помыть посуду


# Задача 2: LRU Cache

In [2]:
from collections import OrderedDict


class LRUCache:
    def __init__(self, capacity):
        self.capacity = capacity
        self.cache_data = OrderedDict()

    @property
    def cache(self):
        if self.cache_data:
            oldest_key = next(iter(self.cache_data))
            return self.cache_data[oldest_key]
        return None

    @cache.setter
    def cache(self, key_value):
        key, value = key_value
        if key in self.cache_data:
            self.cache_data.move_to_end(key)
        self.cache_data[key] = value
        if len(self.cache_data) > self.capacity:
            self.cache_data.popitem(last=False)

    def get(self, key):
        if key in self.cache_data:
            self.cache_data.move_to_end(key)
            return self.cache_data[key]
        return None

    def print_cache(self):
        print("LRU Cache:")
        for key, value in self.cache_data.items():
            print(f"{key} : {value}")


cache = LRUCache(3)

cache.cache = ("key1", "value1")
cache.cache = ("key2", "value2")
cache.cache = ("key3", "value3")

cache.print_cache()

print()
print(cache.get("key2"))
print()

cache.cache = ("key4", "value4")

cache.print_cache()

LRU Cache:
key1 : value1
key2 : value2
key3 : value3

value2

LRU Cache:
key3 : value3
key2 : value2
key4 : value4


# Задача 3: Декоратор кэширования для чисел Фибоначчи

In [3]:
def cache_decorator(func):
    cache = {}

    def wrapper(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result

    return wrapper


@cache_decorator
def fibonacci(n):
    if n <= 1:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)


print(f"fibonacci(10) = {fibonacci(10)}")
print(f"fibonacci(30) = {fibonacci(30)}")
print(f"fibonacci(35) = {fibonacci(35)}")

fibonacci(10) = 55
fibonacci(30) = 832040
fibonacci(35) = 9227465


# Задача 4: Крестики-нолики

In [7]:
class Cell:
    def __init__(self, number):
        self.number = number
        self.is_occupied = False
        self.symbol = ' '

    def occupy(self, symbol):
        if not self.is_occupied:
            self.is_occupied = True
            self.symbol = symbol
            return True
        return False


class Board:
    def __init__(self):
        self.cells = [Cell(i) for i in range(1, 10)]

    def display(self):
        print("\n")
        for i in range(0, 9, 3):
            print(f" {self.cells[i].symbol} | {self.cells[i + 1].symbol} | {self.cells[i + 2].symbol} ")
            if i < 6:
                print("-----------")
        print("\n")

    def make_move(self, cell_number, symbol):
        if 1 <= cell_number <= 9:
            return self.cells[cell_number - 1].occupy(symbol)
        return False

    def check_winner(self):
        win_combinations = [
            [0, 1, 2], [3, 4, 5], [6, 7, 8],
            [0, 3, 6], [1, 4, 7], [2, 5, 8],
            [0, 4, 8], [2, 4, 6]
        ]

        for combo in win_combinations:
            if (self.cells[combo[0]].symbol == self.cells[combo[1]].symbol == self.cells[combo[2]].symbol
                    and self.cells[combo[0]].symbol != ' '):
                return self.cells[combo[0]].symbol
        return None

    def is_full(self):
        return all(cell.is_occupied for cell in self.cells)


class Player:
    def __init__(self, name, symbol):
        self.name = name
        self.symbol = symbol

    def make_move(self, board):
        while True:
            try:
                cell_number = int(input(f"{self.name}, введите номер клетки (1-9): "))
                if board.make_move(cell_number, self.symbol):
                    return
                else:
                    print("Клетка занята или неверный номер. Попробуйте снова.")
            except ValueError:
                print("Введите число от 1 до 9.")


def play_game():
    board = Board()
    player1 = Player("Игрок 1", "X")
    player2 = Player("Игрок 2", "O")
    current_player = player1

    print("Добро пожаловать в игру Крестики-Нолики!")
    print("Нумерация клеток:")
    print(" 1 | 2 | 3 ")
    print("-----------")
    print(" 4 | 5 | 6 ")
    print("-----------")
    print(" 7 | 8 | 9 ")

    while True:
        board.display()
        current_player.make_move(board)

        winner = board.check_winner()
        if winner:
            board.display()
            print(f"Победил {current_player.name} ({winner})!")
            break

        if board.is_full():
            board.display()
            print("Ничья!")
            break

        current_player = player2 if current_player == player1 else player1


play_game()


Добро пожаловать в игру Крестики-Нолики!
Нумерация клеток:
 1 | 2 | 3 
-----------
 4 | 5 | 6 
-----------
 7 | 8 | 9 


   |   |   
-----------
   |   |   
-----------
   |   |   




 X |   |   
-----------
   |   |   
-----------
   |   |   




 X | O |   
-----------
   |   |   
-----------
   |   |   




 X | O |   
-----------
   | X |   
-----------
   |   |   




 X | O | O 
-----------
   | X |   
-----------
   |   |   




 X | O | O 
-----------
   | X |   
-----------
   |   | X 


Победил Игрок 1 (X)!
